#Imports and mounting

In [1]:
!pip install sentence-transformers
import pandas as pd
import json
import random
import csv
from collections import defaultdict
import pickle
import os
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

from sentence_transformers import models, losses, util, SentenceTransformer, SentencesDataset, InputExample, evaluation
from sentence_transformers.cross_encoder import CrossEncoder
from torch import nn
from torch.utils.data import DataLoader
import torch

random.seed(13)






###Reading the android dataset; android_related.json

In [2]:
from google.colab import drive
drive.mount('/gdrive')
folder = '/gdrive/MyDrive/quora_android_2'
os.makedirs(folder,exist_ok = True)
with open('/gdrive/MyDrive/sbert_supervised/android_nosw.json','r') as a:
    qr = pd.read_json(a,orient='table')

Mounted at /gdrive


FileNotFoundError: ignored

###Some Important Functions

In [ ]:
def string_sentence(i,qr):
    return qr.loc[i,'Title']+''.join(qr.loc[i,'Tags']) + qr.loc[i,'Text']

In [ ]:
def both_related(qr):
    for i in qr.index.values:
        if(len(qr.loc[i,'Related'])!=0):
            for j in qr.loc[i,'Related']:
                if(j in qr.index.values):
                    if((i in qr.loc[j,'Related'])==0):
                        qr.loc[j,'Related'].append(i)
    return qr

In [ ]:
qr.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17132 entries, 31 to 219026
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   AcceptedAnswerId  6631 non-null   float64
 1   AnswerCount       17132 non-null  float64
 2   AnswerIds         17132 non-null  object 
 3   Attachments       4246 non-null   object 
 4   OS                4246 non-null   object 
 5   PostTypeId        17132 non-null  int64  
 6   Related           17132 non-null  object 
 7   Score             17132 non-null  int64  
 8   Tags              17132 non-null  object 
 9   Text              17132 non-null  object 
 10  Title             17132 non-null  object 
dtypes: float64(2), int64(2), object(7)
memory usage: 1.6+ MB


In [ ]:
print(torch.cuda.is_available())
# torch.cuda.memory_reserved(0)
!nvidia-smi

True
Sat Jan 23 12:52:23 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    27W / 250W |     10MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+--

#Creating Datas for different objectives (Only Once)

Source File: https://github.com/UKPLab/sentence-transformers/blob/master/examples/training/quora_duplicate_questions/create_splits.py

Objectives: 

1. Classfication   
2. Related Question Mining
3. Information Retreival



In [ ]:
# qr = both_related(qr)
train_qr,dev_qr = train_test_split(qr,test_size=0.02,shuffle = True,random_state=13)
train_qr,test_qr = train_test_split(train_qr,test_size=0.02,shuffle=True,random_state = 13)

###Positive Pair Generation

In [4]:
def pos_pair_gen(qr,qr_index):
    index = 0
    data_pos = list()
    al_pos = list()
    related_not_found = 0
    # score = list()
    for i in qr_index:
        if(len(qr.loc[i,'Related'])!=0):
            for j in qr.loc[i,'Related']:
                if( ( [i,j] in al_pos or [j,i] in al_pos) != 1 ):
                    try:

                        # t1 = string_sentence(i,qr)
                        # t2 = string_sentence(j,qr)
                        data_pos.append([i,j])
                        index+= 1
                        al_pos.append([ i , j ])
                        # score.append(1)
                        # break
                    except:
                        related_not_found+=1
                        # print('Related not found,parent id: ',i,' id:',j,'overall count: ',related_not_found)
    print(len(data_pos))
    return data_pos

###Negative Data Pairs

In [5]:
def neg_pair_gen(qr,num_pairs,al_acc=list()):
    data_neg = list()
    for i in range(0,num_pairs):
        id1 = random.choice(qr.index.values)
        id2 = random.choice(qr.index.values)
        while(id2 in qr.loc[id1,'Related'] or id1 in qr.loc[id2,'Related'] or [id1,id2] in al_acc or [id2,id1] in al_acc):
            id1 = random.choice(qr.index.values)
            id2 = random.choice(qr.index.values)
        # t1 = string_sentence(id1,qr)
        # t2 = string_sentence(id2,qr)
        data_neg.append([id1,id2])
        # score.append(0)
        al_acc.append([id1,id2])
    print(len(data_neg))
    # print(len(score))
    return data_neg,al_acc

###Add transitive closure (if a,b and b,c duplicates => a,c are duplicates) (Not for related)

In [ ]:
def transitive(qr,data_pos,al_pos):
    for i in qr.index.values:
        for j in qr.loc[i,'Related']:
            if(j in qr.index.values):
                for k in qr.loc[j,'Related']:
                    if(k in qr.index.values):
                        if(([i,k] in al_pos or [k,i] in al_pos or i==k) ==0):
                            t1 = qr.loc[i,'Title'] + ''.join(qr.loc[i,'Tags']) + qr.loc[i,'Text']
                            t2 = qr.loc[k,'Title'] + ''.join(qr.loc[k,'Tags']) + qr.loc[k,'Text']
                            data_pos.append([t1,t2])
                            al_pos.append([i,k])
    print(len(data_pos))
    return data_pos
    # with open('/gdrive/MyDrive/quora_android/data_pos.txt','wb') as a:
    #     pickle.dump(data_pos,a)
    #     pickle.dump(al_pos,a)

###Dataset Formation, splitting and saving

In [16]:
# train_data_pos = pos_pair_gen(train_qr,train_qr.index.values)
# train_data_neg,al_acc = neg_pair_gen(train_qr,len(train_data_pos))
dev_data_pos = checker(pd.concat([dev_qr,train_qr]),pos_pair_gen(pd.concat([dev_qr,train_qr]),list(dev_qr.index.values)))
dev_data_neg,al_acc = neg_pair_gen(dev_qr,len(dev_data_pos),al_acc)
test_data_pos = checker(pd.concat([test_qr,train_qr]),pos_pair_gen(pd.concat([test_qr,train_qr]),test_qr.index.values))
test_data_neg,al_acc = neg_pair_gen(test_qr,len(test_data_pos),al_acc)

2304
598 1706
1706
2224
535 1689
1689


In [8]:
def checker(qr,qr_index):
    new_idx = list()
    removed_pairs = 0
    for i,j in qr_index:
        if((i in qr.index.values) and (j in qr.index.values)):
            new_idx.append([i,j])
        else:
            removed_pairs +=1
    
    print(removed_pairs,len(new_idx))
    return new_idx

In [ ]:
os.makedirs(folder+'/data/splits',exist_ok=True)
with open(folder+'/data/splits/pandas_split.txt','wb') as a:
    pickle.dump(train_qr,a)
    pickle.dump(dev_qr,a)
    pickle.dump(test_qr,a)

In [17]:
with open('/gdrive/MyDrive/MultiModal'+'/data/splits/data_pos_neg.txt','wb') as a:
    pickle.dump(train_data_pos,a)
    pickle.dump(train_data_neg,a)
    pickle.dump(dev_data_pos,a)
    pickle.dump(dev_data_neg,a)
    pickle.dump(test_data_pos,a)
    pickle.dump(test_data_neg,a)
    
    # For next step, save and load al_acc for paraphase mining negative pairs

In [ ]:
# with open(folder+'/data/android.txt','rb') as a:
#     data_pos = pickle.load(a)
#     al_pos = pickle.load(a)
#     data_neg = pickle.load(a)
#     score = pickle.load(a)

with open(folder+'/data/splits/data_pos_neg.txt','rb') as a:
    train_data_pos = pickle.load(a)
    train_data_neg = pickle.load(a)
    dev_data_pos = pickle.load(a)
    dev_data_neg = pickle.load(a)
    test_data_pos = pickle.load(a)
    test_data_neg = pickle.load(a)

In [ ]:
# data = data_pos + data_neg
# data,score = shuffle(data,score)
# train_data,dev_data,train_score,dev_score = train_test_split(data,score,test_size=0.15,random_state=13)
# test_data,dev_data,test_score,dev_score = train_test_split(dev_data,dev_score,test_size=0.33,random_state=13)

train_score = np.ones(2*len(train_data_pos)).tolist()
for i in range(len(train_data_pos),2*len(train_data_pos)):
    train_score[i] = 0
train_data = train_data_pos + train_data_neg
train_data,train_score = shuffle(train_data,train_score)

dev_score = np.ones(2*len(dev_data_pos)).tolist()
for i in range(len(dev_data_pos),2*len(dev_data_pos)):
    dev_score[i] = 0
dev_data = dev_data_pos + dev_data_neg
dev_data,dev_score = shuffle(dev_data,dev_score)

test_score = np.ones(2*len(test_data_pos)).tolist()
for i in range(len(test_data_pos),2*len(test_data_pos)):
    test_score[i] = 0
test_data = (test_data_neg) + test_data_pos
test_data,test_score = shuffle(test_data,test_score)

In [ ]:
print(train_score[:10])

[0, 0, 1.0, 0, 1.0, 1.0, 1.0, 1.0, 0, 1.0]


In [ ]:
with open(folder+'/data/splits/train_dev_test.txt','wb') as a:
    pickle.dump(train_data,a)
    pickle.dump(train_score,a)
    pickle.dump(dev_data,a)
    pickle.dump(dev_score,a)
    pickle.dump(test_data,a)
    pickle.dump(test_score,a)

#Sentence Transformer (For Training, start from here)

###Loading Data

In [3]:
with open(folder+'/data/splits/pandas_split.txt','rb') as a:
    train_qr=pickle.load(a)
    dev_qr = pickle.load(a)
    test_qr = pickle.load(a)

In [ ]:
with open(folder+'/data/splits/train_dev_test.txt','rb') as a:
    train_data=pickle.load(a)
    train_score = pickle.load(a)
    dev_data = pickle.load(a)
    dev_score = pickle.load(a)
    test_data = pickle.load(a)
    test_score = pickle.load(a)

##Model

###Creating Model

In [ ]:
# For model from HuggingFace Eg: BERT/ROberta etc.
word_embedding_model = models.Transformer('distilbert-base-uncased',max_seq_length=512)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())

model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

# #For Pretrrained Sentence-Transformers, downloaded from the github of sentence-transformers
# model = SentenceTransformer('stsb-distilbert-base')

####Models Parameters

In [ ]:
batch_size = 8
num_epochs = 1
os.makedirs(folder+'/bert',exist_ok=True)
model_path = folder+'/bert'

In [ ]:
!nvidia-smi

Sat Jan 23 12:53:16 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    27W / 250W |     10MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

###OnlineConstrative Loss

In [ ]:
distance_metric = losses.SiameseDistanceMetric.COSINE_DISTANCE
margin = 0.5

train_constrantiveloss = DataLoader(train_examples,batch_size=batch_size,shuffle=True)
train_loss_ConstrativeLoss = losses.OnlineContrastiveLoss(model=model, distance_metric=distance_metric, margin=margin)

###MultipleNegativeRanking Loss

In [ ]:
train_samples_MNRloss = list()
for i in range(0,len(train_data_pos)):
    train_samples_MNRloss.append(InputExample(texts=[train_data_pos[i][0],train_data_pos[i][1]],label=1))
    train_samples_MNRloss.append(InputExample(texts=[train_data_pos[i][1],train_data_pos[i][0]],label=1))

train_MNRloss = DataLoader(train_samples_MNRloss,batch_size=batch_size,shuffle=True)
train_loss_MNRloss = losses.MultipleNegativesRankingLoss(model)

###Evaluators

There are three evaluators mentioned in training_multi-task-learning.py

1.   BinaryClassification Evaluator
2.   Paraphase Mining (equivalent to top k related but for small number of queries)
3.   Information retreival(equivalent to top k related but for a larger number of querires; Use a bi encoder and cross encoder for efficient and fast retreival)

Both paraphase mining and information retrieval evaluators uses metrics such as Mean Reciprocal Rank (MRR), Recall@k, and Normalized Discounted Cumulative Gain (NDCG)

** **For now only implementing binary classifier evaluator** **



In [ ]:
evaluator_dev = list()
evaluator_test = list()

In [ ]:
def corpus(qr):
    corpus = dict()
    for i in qr.index.values:
        corpus[i] = string_sentence(i,qr)
    return corpus

####Binary Evaluator

In [ ]:
evaluator_b_dev = evaluation.BinaryClassificationEvaluator(dev_s1,dev_s2,dev_score,show_progress_bar=True)
evaluator_b_test = evaluation.BinaryClassificationEvaluator(test_s1, test_s2, test_score,show_progress_bar=True)
evaluator_dev.append(evaluator_b_dev)
evaluator_test.append(evaluator_b_test)

####Paraphrase Mining Evaluator

In [ ]:
def paraphrase_dataset(index,qr,qr_orig):
    # Currently the code is for positive pairs only  
    duplicates_list = list()
    duplicates_dict = dict()
    for i in index:
        if(len(qr_orig.loc[i,'Related'])!=0):
            for j in qr_orig.loc[i,'Related']:
                if(j in qr.index.values):
                    duplicates_list.append([i,j])
                    # if(i in duplicates_dict_dev):
                    #     duplicates_dict_dev[i][j] = True
                    # else:
                    #     duplicates_dict_dev[i] = dict()
                    #     duplicates_dict_dev[i][j] = True #For negative pairs, create dataset again and save al_acc
    return duplicates_list

In [ ]:
## For dev dataset
duplicates_list_dev = paraphrase_dataset(dev_qr.index.values,train_qr,qr) 
print(len(duplicates_list_dev))

## For test dataset
duplicates_list_test = paraphrase_dataset(test_qr.index.values,train_qr,qr)
print(len(duplicates_list_test))

1665
1657


In [ ]:
# evaluator_p_dev = evaluation.ParaphraseMiningEvaluator(corpus(train_qr),duplicates_list_dev,show_progress_bar=True)
# evaluator_p_test = evaluation.ParaphraseMiningEvaluator(corpus(train_qr),duplicates_list_test,show_progress_bar=True)
# evaluator_dev.append(evaluator_p_dev)
# evaluator_test.append(evaluator_p_test)

####Information Retreival Evaluator

In [ ]:
def info_ret(index,qr,qr_orig,limit):
    queries = dict()
    rel_docs = dict()
    cnt = 0
    done = list()
    lim_index = 0
    for j in index:
        if(len(qr_orig.loc[j,'Related'])!=0):
            for k in qr_orig.loc[j,'Related']:
                if(k in qr.index.values):
                    lim_index+=1
                    break
    if(lim_index<limit):
        limit = lim_index
    for i in range(0,limit):
        id = random.choice(index)
        while(id in done):
            id = random.choice(index)
        if(len(qr_orig.loc[id,'Related'])!=0):
            for j in qr_orig.loc[id,'Related']:
                if(j in qr.index.values):
                    queries[id] = string_sentence(id,qr_orig)
                    done.append(id)
                    if(id in rel_docs):
                        rel_docs[id].append(j)
                    else:
                        rel_docs[id] = list()
                        rel_docs[id].append(j)
        cnt+=1
    return queries,rel_docs

In [ ]:
# #For dev dataset
# queries_dev,rel_docs_dev = info_ret(dev_qr.index.values,train_qr,qr,100)
# #For test dataset
# queries_test,rel_docs_test = info_ret(test_qr.index.values,train_qr,qr,100)
# #For train
# queries_train,rel_docs_train = info_ret(train_qr.index.values,train_qr,qr,100)

In [ ]:
# with open(folder+'/devinfo_100.txt','wb') as a:
#     pickle.dump(queries_dev,a)
#     pickle.dump(rel_docs_dev,a)

# with open(folder+'/testinfo_100.txt','wb') as a:
#     pickle.dump(queries_test,a)
#     pickle.dump(rel_docs_test,a)

# with open(folder+'/traininfo_100.txt','wb') as a:
#     pickle.dump(queries_train,a)
#     pickle.dump(rel_docs_train,a)

In [ ]:
with open(folder+'/devinfo_100.txt','rb') as a:
    queries_dev = pickle.load(a)
    rel_docs_dev= pickle.load(a)

with open(folder+'/testinfo_100.txt','rb') as a:
    queries_test= pickle.load(a)
    rel_docs_test= pickle.load(a)

with open(folder+'/traininfo_100.txt','rb') as a:
    queries_train= pickle.load(a)
    rel_docs_train= pickle.load(a)

In [ ]:
evaluator_i_dev = evaluation.InformationRetrievalEvaluator(queries_dev,corpus(train_qr),rel_docs_dev,show_progress_bar=True)
evaluator_i_test = evaluation.InformationRetrievalEvaluator(queries_test,corpus(train_qr),rel_docs_test,show_progress_bar=True)
evaluator_i_train = evaluation.InformationRetrievalEvaluator(queries_train,corpus(train_qr),rel_docs_train,show_progress_bar=True)
evaluator_dev.append(evaluator_i_dev)
evaluator_test.append(evaluator_i_test)

####Sequential Evaluator

Combining all the three evaluators mentioned above

In [ ]:
seq_evaluator_dev = evaluation.SequentialEvaluator(evaluator_dev, main_score_function=lambda scores: scores[-1])
seq_evaluator_test = evaluation.SequentialEvaluator(evaluator_test, main_score_function=lambda scores: scores[-1])

Evaluating Model without training

In [ ]:
# seq_evaluator_dev(model, epoch=0, steps=0, output_path=model_path)
# seq_evaluator_test(model, epoch=0, steps=0, output_path=model_path)

###Fitting Model

In [ ]:
import math
warmup_steps = math.ceil(len(train_examples) * num_epochs / batch_size * 0.1) #10% of train data for warm-up

In [ ]:
print(torch.cuda.is_available())

True


In [ ]:
x = list()
y = list()
def callback_model(score,epoch,steps):
    if(steps==-1):
        steps=0
        epoch+=1
    x.append(float(epoch + steps/89590))
    y.append(float(score))
    print('Score after epoch ', str(epoch),' and steps ', str(steps),'is ',str(score))

In [ ]:
model.fit(train_objectives=[(train_constrantiveloss,train_loss_ConstrativeLoss),(train_MNRloss,train_loss_MNRloss)],
          evaluator=seq_evaluator_dev,
          epochs=num_epochs,
          evaluation_steps=4000,
          warmup_steps = warmup_steps,
          output_path=model_path,
          output_path_ignore_not_empty=True,
          use_amp = True,
          callback = callback_model
          )

Corpus Chunks:   0%|          | 0/1 [00:00<?, ?it/s]

Corpus Chunks: 100%|██████████| 1/1 [01:28<00:00, 88.02s/it]


Score after epoch  0  and steps  4000 is  0.16666497671375907


Corpus Chunks:   0%|          | 0/1 [00:00<?, ?it/s]

Corpus Chunks: 100%|██████████| 1/1 [01:28<00:00, 88.25s/it]


...

Corpus Chunks: 100%|██████████| 1/1 [01:28<00:00, 88.41s/it]


Score after epoch  0  and steps  16000 is  0.16266672523028136



Corpus Chunks:   0%|          | 0/1 [00:00<?, ?it/s]

Corpus Chunks: 100%|██████████| 1/1 [01:28<00:00, 88.79s/it]

Score after epoch  1  and steps  0 is  0.16678726292834603



Plots

In [ ]:
# %matplotlib inline
# from matplotlib import pyplot as plt
# plt.plot(x,y)
# plt.show()

# os.makedirs(folder+'/plots',exist_ok=True)
# plt.savefig(folder+'/plots/accuarcy.png')
# with open(folder+'/plots/plot_data.txt','wb') as f:
#     pickle.dump(x,f)
#     pickle.dump(y,f)

#Testing

###Loading Models

In [ ]:
os.makedirs(folder+'/test/pretrained',exist_ok=True)
os.makedirs(folder+'/test/final_3',exist_ok=True)
os.makedirs(folder+'/test/sbert_naive',exist_ok=True)
os.makedirs(folder+'/test/quora_we',exist_ok=True)

os.makedirs(folder+'/dev_t/pretrained',exist_ok=True)
os.makedirs(folder+'/dev_t/final_3',exist_ok=True)
os.makedirs(folder+'/dev_t/sbert_naive',exist_ok=True)
os.makedirs(folder+'/dev_t/quora_we',exist_ok=True)

os.makedirs(folder+'/train_t/pretrained',exist_ok=True)
os.makedirs(folder+'/train_t/final_3',exist_ok=True)
os.makedirs(folder+'/train_t/sbert_naive',exist_ok=True)
os.makedirs(folder+'/train_t/quora_we',exist_ok=True)

model_pre = SentenceTransformer('stsb-distilbert-base')
model_final = SentenceTransformer('/gdrive/MyDrive/final_model_3')
model_unk = SentenceTransformer('//gdrive/MyDrive/sbert_supervised')
model_quora_we = SentenceTransformer(folder+'/distilbert_quora')

In [ ]:
evaluator_i_train(model_pre, epoch=0, steps=0, output_path=folder+'/train_t/pretrained')
evaluator_i_train(model_final, epoch=0, steps=0, output_path=folder+'/train_t/final_3')
evaluator_i_train(model_unk, epoch=0, steps=0, output_path=folder+'/train_t/sbert_naive')
evaluator_i_train(model_quora_we, epoch=0, steps=0, output_path=folder+'/train_t/quora_we')

In [ ]:
seq_evaluator_dev(model_pre, epoch=0, steps=0, output_path=folder+'/dev_t/pretrained')
seq_evaluator_dev(model_final, epoch=0, steps=0, output_path=folder+'/dev_t/final_3')
seq_evaluator_dev(model_unk, epoch=0, steps=0, output_path=folder+'/dev_t/sbert_naive')
seq_evaluator_dev(model_quora_we, epoch=0, steps=0, output_path=folder+'/dev_t/quora_we')

In [ ]:
seq_evaluator_test(model_pre, epoch=0, steps=0, output_path=folder+'/test/pretrained')
seq_evaluator_test(model_final, epoch=0, steps=0, output_path=folder+'/test/final_3')
seq_evaluator_test(model_unk, epoch=0, steps=0, output_path=folder+'/test/sbert_naive')
seq_evaluator_test(model_quora_we, epoch=0, steps=0, output_path=folder+'/test/quora_we')

In [ ]:
os.makedirs(folder+'/test',exist_ok=True)
seq_evaluator_test(model,output_path=folder+'/test')


Corpus Chunks:   0%|          | 0/1 [00:00<?, ?it/s]


Corpus Chunks: 100%|██████████| 1/1 [00:56<00:00, 56.18s/it]


0.2072731859650183

In [ ]:
os.makedirs(folder+'/train',exist_ok=True)
evaluator_i_train(model,output_path=folder+'/train')


Corpus Chunks:   0%|          | 0/1 [00:00<?, ?it/s]


Corpus Chunks: 100%|██████████| 1/1 [00:56<00:00, 56.06s/it]


0.16333331893557396